<a href="https://colab.research.google.com/github/narsym/deep-learning-with-tensorflow-2.0/blob/master/Using_BERT_as_a_part_of_your_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports 

In [0]:
!pip install transformers

     |████████████████████████████████| 675kB 4.6MB/s 
     |████████████████████████████████| 890kB 19.3MB/s 
     |████████████████████████████████| 3.8MB 17.0MB/s 
     |████████████████████████████████| 1.1MB 45.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a3a5824bd9ccd661dfd7d82f34ec076ccc8aea5f6f4628f17ac4dcf67cbadfb4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import os
import tensorflow as tf
import tensorflow_datasets
from transformers import BertTokenizer, TFBertForSequenceClassification, BertForSequenceClassification, glue_convert_examples_to_features

Few constants to use later

In [0]:
BATCH_SIZE = 32
FINE_TUNED_MODEL_DIR = './data'

Instantiate tokenizer and model to wrap transformers library 

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')

Load training and validation data from the tensorflow_datasets package

In [0]:
data, info = tensorflow_datasets.load(
    'glue/mrpc', with_info = True
)
num_train = info.splits['train'].num_examples
num_valid = info.splits['validation'].num_examples

INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for glue/mrpc/1.0.0
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.location from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset glue (/root/tensorflow_datasets/glue/mrpc/1.0.0)


INFO:absl:Downloading https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc into /root/tensorflow_datasets/downloads/fire.goog.com_v0_b_mtl-sent-repr.apps.com_o_2FjSIMlCiqs1QSmIykr4IRPnEHjPuGwAz5i40v8K9U0Z8.tsvalt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc.tmp.6aa7673646ee4fb49d1955d8b7309399...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_trainfGxPZuQWGBti4Tbd1YNOwQr-OqxPejJ7gcp0Al6mlSk.txt.tmp.b7a5e9fbb54847a0bbd025c81c012350...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_test0PdekMcyqYR-w4Rx_d7OTryq0J3RlYRn4rAMajy9Mak.txt.tmp.8566b9d4e58c4ba1ab82f65667ea87af...
/usr/local/lib/python3.6/dist-packages/urllib3/co

Shuffling and writing examples to /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteBNRBFV/glue-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteBNRBFV/glue-train.tfrecord. Shard lengths: [3668]
INFO:absl:Generating split validation


Shuffling and writing examples to /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteBNRBFV/glue-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteBNRBFV/glue-validation.tfrecord. Shard lengths: [408]
INFO:absl:Generating split test


Shuffling and writing examples to /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteBNRBFV/glue-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteBNRBFV/glue-test.tfrecord. Shard lengths: [1725]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.AUTO.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/glue/mrpc/1.0.0


Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/mrpc/1.0.0. Subsequent calls will reuse this data.


Prepare dataset for GLUE as tf.data.Dataset instance

In [0]:
Xtrain = glue_convert_examples_to_features(data['train'], tokernizer, 128, 'mrpc')
Xtrain = Xtrain.shuffle(128).batch(BATCH_SIZE).repeat(-1)
Xvalid = glue_convert_examples_to_features(data['validation'], tokernizer, 128, 'mrpc')
Xvalid = Xvalid.shuffle(128).batch(BATCH_SIZE).repeat(-1)

Optimizer, loss, metrics and fine tuning

In [0]:
opt = tf.keras.optimizers.Adam(learning_rate = 3e-5, epsilon = 1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
metrics = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer = opt, loss = loss, metrics = [metrics])

In [0]:
train_steps = num_train // BATCH_SIZE
valid_steps = num_valid // BATCH_SIZE
history = model.fit(Xtrain, epochs=2, steps_per_epoch=train_steps, validation_data=Xvalid, validation_steps=valid_steps)

Epoch 1/2
114/114 [==============================] - 58s 511ms/step - loss: 0.5922 - accuracy: 0.6891 - val_loss: 0.4737 - val_accuracy: 0.7630
Epoch 2/2
114/114 [==============================] - 56s 491ms/step - loss: 0.4007 - accuracy: 0.8256 - val_loss: 0.3777 - val_accuracy: 0.8359


Save our fine tuned model

In [0]:
model.save_pretrained(FINE_TUNED_MODEL_DIR)

To predict that pair of sentences as paraphrases of one another we load the model as pytorch model as tensorflow does not support it now.

In [0]:
saved_model = BertForSequenceClassification.from_pretrained(FINE_TUNED_MODEL_DIR, from_tf = True)# tf check point

Test our model on sentences

In [0]:
def print_result(id1, id2, pred):
  if pred == 1:
    print('sentence1 is a paraphrase of sentence0')
  else:
    print('sentence1 is not a paraphrase of sentence0')
  
sentence_0 = "At least 12 people were killed in the battle last week."
sentence_1 = "At least 12 people lost their lives in last weeks fighting."
sentence_2 = "The fires burnt down the houses on the street."

inputs1 = tokenizer.encode_plus(sentence_0, sentence_1, add_special_tokens = False, return_tensors = 'pt')
inputs2 = tokenizer.encode_plus(sentence_0, sentence_2, add_special_tokens = False, return_tensors = 'pt')

pred_1 = saved_model(**inputs1)[0].argmax().item()
pred_2 = saved_model(**inputs2)[0].argmax().item()

print_result(0, 1, pred_1)
print_result(0, 2, pred_2)

sentence1 is a paraphrase of sentence0
sentence1 is not a paraphrase of sentence0


So, that is the result.